In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_dir = 'seefood/train'
test_dir = 'seefood/test'

train_data = datasets.ImageFolder(root=train_dir, transform=transform)
test_data = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

In [4]:
image, label = train_data[0]
image.size()

torch.Size([3, 224, 224])

In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 53 * 53, 128)
        self.fc2 = nn.Linear(128, 2)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [6]:
cnn_model = Model()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_model.parameters(), lr=0.01, momentum=0.9)

In [7]:
for epoch in range(15):
    print(f'Training epoch: {epoch}')
    
    rn_loss = 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = cnn_model(inputs)
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        rn_loss += loss.item()
    print(f'Loss: {rn_loss / len(train_loader):.4f}')

Training epoch: 0
Loss: 0.6941
Training epoch: 1
Loss: 0.6436
Training epoch: 2
Loss: 0.6481
Training epoch: 3
Loss: 0.5858
Training epoch: 4
Loss: 0.5877
Training epoch: 5
Loss: 0.6580
Training epoch: 6
Loss: 0.6233
Training epoch: 7
Loss: 0.5155
Training epoch: 8
Loss: 0.4248
Training epoch: 9
Loss: 0.2934
Training epoch: 10
Loss: 0.1805
Training epoch: 11
Loss: 0.1719
Training epoch: 12
Loss: 0.1018
Training epoch: 13
Loss: 0.0628
Training epoch: 14
Loss: 0.0470


In [8]:
#torch.save(cnn_model.state_dict(), 'train_net.pth')

In [9]:
correct = 0
total = 0

cnn_model.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = cnn_model(images)
        
        _, predictions = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy}')

Accuracy: 56.8


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

cnn_model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = cnn_model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.numpy())
        all_labels.extend(preds.numpy())
        
acc = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 56.8000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
